In [1]:
import bw2data as bd
import bw2calc as bc

In [3]:
bd.projects.set_current("bw25_plca_grid_expansion")

## Components to materials


In [4]:
bd.databases

Databases dictionary with 45 objects, including:
	ecoinvent-3.10-biosphere
	ecoinvent-3.10-cutoff
	ei310_SSP2_Base_2023
	ei310_SSP2_Base_2025
	ei310_SSP2_Base_2030
	ei310_SSP2_Base_2035
	ei310_SSP2_Base_2037
	ei310_SSP2_Base_2040
	ei310_SSP2_Base_2045
	ei310_SSP2_RCP19_2023
Use `list(this object)` to get the complete list.

In [5]:
status_quo_node = bd.get_node(database="grid_status_quo", name="grid_status_quo")
fu = {status_quo_node: 1}

In [6]:
method = (
    "EF v3.1 no LT",
    "climate change no LT",
    "global warming potential (GWP100) no LT",
)

In [7]:
component_scores = {}
lca = bc.LCA(fu, method=method)
lca.lci(factorize=True)
for component in status_quo_node.technosphere():
    material_scores = {}
    for material in component.input.technosphere():
        lca.lcia(demand={material.input.id: material.amount * component.amount})
        material_scores[material.input["name"]] = lca.score
    component_scores[component.input["name"]] = material_scores

/Users/timodiepers/anaconda3/envs/plca-grid-expansion/lib/python3.11/site-packages/bw2calc/lca_base.py:127: SparseEfficiencyWarning: splu converted its input to CSC format
  self.solver = factorized(self.technosphere_matrix)
/Users/timodiepers/anaconda3/envs/plca-grid-expansion/lib/python3.11/site-packages/scikits/umfpack/umfpack.py:736: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 5.51e+12)
  warnings.warn(msg, UmfpackWarning)


In [8]:
import pandas as pd

df = pd.DataFrame(component_scores)

In [9]:
def map_labels(labels, groups, other_label="other"):
    new_labels = {}
    for label in labels:
        for key, group in groups.items():
            if key in label:
                new_labels[label] = group
                break
        else:
            new_labels[label] = other_label
    return new_labels

In [10]:
component_groups = {
    "Overhead line": "overhead lines",
    "cable": "cables",
    "Transformer": "transformers",
    "switchgear": "switchgears",
    "Substation": "substations",
}

material_groups = {
    "aluminium": "aluminium",
    "copper": "copper",
    "iron": "iron & steel",
    "steel": "iron & steel",
    "concrete": "concrete",
    "cement": "concrete",
    "sulfur hexafluoride": "SF6",
    "polyethylene": "plastics",
    "polypropylene": "plastics",
    "plastic": "plastics",
}

df.index = df.index.map(
    map_labels(df.index, material_groups, other_label="other materials")
)
df.columns = df.columns.map(map_labels(df.columns, component_groups))

df_components_to_materials = df.groupby(level=0).sum().T.groupby(level=0).sum().T
df_components_to_materials

,cables,overhead lines,substations,switchgears,transformers
SF6,0.000000e+00,0.000000e+00,0.000000e+00,5.099408e+08,0.000000e+00
aluminium,1.830497e+10,2.059093e+10,0.000000e+00,3.826334e+08,9.588930e+08
concrete,0.000000e+00,2.888274e+09,2.168151e+08,0.000000e+00,0.000000e+00
copper,3.146513e+09,1.596225e+09,0.000000e+00,5.137299e+07,5.668277e+08
iron & steel,4.480560e+08,1.074045e+10,8.913534e+08,4.292127e+07,1.609925e+09
other materials,4.148054e+08,6.033488e+08,0.000000e+00,5.969894e+07,7.023182e+08
plastics,5.493475e+09,8.932584e+06,0.000000e+00,0.000000e+00,0.000000e+00


## Materials to processes


In [10]:
aggregated_material_names = [
    "aggregated material: aluminium",
    "aggregated material: steel",
    "aggregated material: concrete",
    "aggregated material: copper",
    "aggregated material: plastics",
    "aggregated material: other",
]

aggregated_material_nodes = [
    bd.get_node(name=name) for name in aggregated_material_names
]

In [11]:
import bw2analyzer as ba

In [14]:
lca = bc.LCA(
    {aggregated_material_nodes[0]: 1}, method=method
)  # just to build the matrices
lca.lci(factorize=True)
material_top_processes = {}
for mat in aggregated_material_nodes:
    lca.lcia(demand={mat.id: 1})
    top_processes = ba.ContributionAnalysis().annotated_top_processes(lca, limit=5000)
    score_and_product = {}
    for process in top_processes:
        if process[2]["reference product"] in score_and_product:
            score_and_product[process[2]["reference product"]] += process[0]
        else:
            score_and_product[process[2]["reference product"]] = process[0]
    material_top_processes[mat["name"]] = score_and_product

In [15]:
df_aggregated_materials = pd.DataFrame(material_top_processes)

In [16]:
product_groups = {
    "electricity": "electricity",
    "heat": "heat",
    "transport": "transport",
    "aluminium": "aluminium (process emissions)",
    "iron": "iron & steel (process emissions)",
    "steel": "iron & steel (process emissions)",
    "coal": "coal",
    "coke": "coal",
    "clinker": "clinker",
    "diesel": "transport",
    "Gas insulated switchgear": "SF6",
    "sulfur hexafluoride": "SF6",
}

df_aggregated_materials.index = df_aggregated_materials.index.map(
    map_labels(
        df_aggregated_materials.index, product_groups, other_label="other processes"
    )
)
df_aggregated_materials.columns = df_aggregated_materials.columns.map(
    map_labels(
        df_aggregated_materials.columns, material_groups, other_label="other materials"
    )
)

df_materials_to_processes = (
    df_aggregated_materials.groupby(level=0).sum().T.groupby(level=0).sum().T
)
df_materials_to_processes

,aluminium,concrete,copper,iron & steel,other materials,plastics
SF6,1.287374e+06,1.252483e+04,1.063052e+05,4.860647e+04,4.897169e+08,5.638800e+04
aluminium (process emissions),4.294168e+09,2.389174e+06,2.435465e+07,4.956728e+06,2.351635e+06,4.676792e+06
clinker,7.654752e+07,1.848610e+09,5.815870e+07,2.106304e+08,2.020967e+07,2.645292e+07
coal,2.583802e+09,7.356678e+07,2.691945e+08,2.052791e+09,6.180054e+07,3.050509e+08
electricity,1.655365e+10,3.468072e+08,2.466410e+09,1.614116e+09,4.899204e+08,1.427320e+09
heat,5.792224e+09,9.315086e+07,6.276506e+08,4.877937e+08,3.008179e+08,3.005004e+08
iron & steel (process emissions),1.117839e+08,2.032110e+07,5.635602e+07,5.652009e+09,2.087370e+07,5.153818e+07
other processes,1.602395e+09,2.114111e+08,7.600215e+08,8.909393e+08,6.667555e+08,2.931285e+09
transport,1.088980e+09,4.126682e+08,4.731807e+08,9.297553e+08,1.914674e+08,3.251729e+08


In [17]:
df_materials_to_processes.aluminium.sum()

32104835499.767765

## Prepare sankey diagram


In [18]:
sankey_data = []  # will be filled with tuples of (source, target, value)

Total impact:


In [19]:
df_components_to_materials.sum().sum()

59308212522.548836

Grid to components:


In [20]:
for component, score in df_components_to_materials.sum().items():
    sankey_data.append((component, "grid status quo", score))

Components to materials:


In [21]:
for target, values in df_components_to_materials.items():
    for source, value in values.items():
        if value != 0:
            sankey_data.append((source, target, value))

Materials to processes:


In [22]:
for target, values in df_materials_to_processes.items():
    for source, value in values.items():
        if value != 0:
            sankey_data.append((source, target, value))

Create a df and add direct emissions


In [32]:
df_sankey = pd.DataFrame(sankey_data, columns=["source", "target", "value"])

In [33]:
df_sankey.value = df_sankey.value / 1e9

In [34]:
df_sankey.columns = ["source", "target", "weight"]

In [38]:

df_sankey.to_csv("sankey_data.csv", index=False)

In [1]:
import pandas as pd

In [2]:
df_sankey = pd.read_csv("sankey_data.csv")

In [4]:
from d3blocks import D3Blocks
d3 = D3Blocks()
d3.sankey(df_sankey)
d3.show()

In [5]:
d3.show(filepath="sankey.html")

The final figure for the paper was created with javascript directly. The code is available on Observable: https://observablehq.com/d/daf3d94f3654d166